https://www.kaggle.com/c/morecomplicatedsentiment/overview

**Заказчику нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, оценить возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных не предоставляется.**

**В соревновании предстоит посмотреть на предоставленные заказчиком отзывы, собрать похожие отзывы в качестве обучающей выборки, и поэкспериментировать с постановкой задачи (разметкой выборки на позитивные и негативные примеры) так, чтобы результат на примерах заказчика был по возможности получше.**


In [1]:
import requests
import time
import re
import json
import pandas as pd
from bs4 import BeautifulSoup as bs

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC


In [2]:
#загрузим имеющиеся данные 
with open('/Users/erikmartirosov/python projects/_Yandex_DS/6_final/simplesentiment2/test.csv', 'r') as tt:
    text = tt.read()
    parser = bs(text, 'html.parser')
    comments = parser.findAll('review')
    
    

In [3]:
#поместим очищенные от разметки данные в DataFrame
df_comments = pd.DataFrame([i.text.replace('\n', ' ') for i in comments], columns=['text'])

In [4]:
#посмотрим на комментарии - речь про мобильные телефоны, поэтому нужно спрасить соответствующие тематике отзывы 
df_comments['text'][13]

'Звук , 2 sim, удобная удобная qwerty клавиатура ,Внешний дизайн телефона Если телефон живет у меня больше года, то это во истину отличный телефон) Когда у меня сломался мой сенсорный телефончик, я не знала что делать. Дел в том, что я не знала, какой новый телефон мне нужно. Вроде не хотелось больше сенсорный, но на обычном стандартном телефоне очень долго и неудобно писать сообщения, но так как в соц. сетях я провожу все время, это было неудобно для меня. Мой взгляд пал на этот прекрасный телефончик.Большое количество интересных цветов свело меня с ума) Так как я человек изменчивый, и яркий салатовый цвет потом мог бы меня угнетать, я решила взять нейтральный черный, и не пожалела) Началась зима, и меня поглотила депрессия, яркий цвет меня бы добил. В соц. сетях я провожу весь день, выхожу только на ночь, но при этом телефону уже год, но он не зависает и не тормозит) Для камеры на 2.0 мп фотографии получаются хорошими и четкими. Динамик тоже очень хороший, в сравнении с моим предыдущ

**Для формирования обучающего набора, воспользуемя api mail.ru:**

вид ссылки на страницу с комментариями на мобильный телефон:

https://hi-tech.mail.ru/ajax/14580352-catalog/feedback/list/?page=1 


In [ ]:
# спарсим список id телефонов с первых 30 страниц сайта
# с помощю регулярного выражения вытащим 8 подряд идущих цифр - это и есть id 
# список с id пригодится для формирования ссылок на страницы с отзывами
phones_id = []

for i in range(1,31):
    req = requests.get(f'https://hi-tech.mail.ru/mobile-catalog/?page={i}')
    parser = bs(req.text, 'html')
    for i in parser.find_all('a', class_='p-catalog-card__link link-holder'):
        try:
            phones_id.append(re.findall(r'\d{8}',i.get('href'))[0])
        except:
            pass
    time.sleep(1) #тайм-аут, чтоб не поймать бан

In [ ]:
len(phones_id) # имеем список из 1426 телефонов

In [7]:
# сделаем запрос и посмотрим на содержимое страницы
req = requests.get(f'https://hi-tech.mail.ru/ajax/{phones_id[120]}-catalog/feedback/list/?page={1}')
parser = bs(req.text, 'html')

In [8]:
newDictionary=json.loads(parser.text)

In [9]:
# содеждание отзывов под тегом data
# всего 5 комментов на странице (0-4)
# 'star' - содержит оценку 
# 'text' - содержит 3 подтипа комментариев (плюсы, минусы, впечатления)
newDictionary['data'][0]['star']

5

In [9]:
# соберем оценки и комментарии в списки
labels = []
comments = []


In [10]:
def get_page(phone_id, page_id):
    req = requests.get(f'https://hi-tech.mail.ru/ajax/{phone_id}-catalog/feedback/list/?page={page_id}')
    parser = bs(req.text, 'html')
    return json.loads(parser.text)

In [11]:
def get_comments(page, labels, comments, n_comments=5, n_types=3):
    for i in range(n_comments):
        try:
            label = page['data'][i]['star']
            comment = [page['data'][i]['text'][j]['text'] for j in range(n_types)]
            
            labels.append(label)
            comments.append(comment)
        except: break

    
    

In [13]:
# пройдемся по нескольким страницам отзывов для каждого телефона и соберем данные 
for i in phones_id:
    for j in range(1,10):
        try:
            page = get_page(i,j)
            get_comments(page,labels, comments)
        except: break

In [14]:
len(labels)

3303

In [15]:
len(comments)

3303

In [16]:
df = pd.DataFrame({'comment':comments,'label':labels})

In [17]:
# будем считать, что все меньше 5 - негативный отзыв
df['label'].replace([1,2,3,4],0, inplace=True)
df['label'].replace(5,1, inplace=True)


In [18]:
# отдельные столбцы: плюсы+коммент и минусы+коммент

df['advantages'] = df['comment'].apply(lambda x : x[0]) +'.' + df['comment'].apply(lambda x : x[2])
df['disadvantages'] = df['comment'].apply(lambda x : x[1]) +'.' + df['comment'].apply(lambda x : x[2])


In [19]:
df.head()

,comment,label,advantages,disadvantages
0,"[Все стандартно внешний вид , экран, производи...",0,"Все стандартно внешний вид , экран, производит...","Нет системы позиционирования бейдоу, хотя она ..."
1,"[мало очень , нагревается и камера не работает...",0,"мало очень .неоднозначные,много ошибок",нагревается и камера не работает .неоднозначны...
2,"[Плюсов нету,греется и плохо ловит антенна, ба...",0,"Плюсов нету,греется и плохо ловит антенна, бат...","Очень дорого и не стоят этих денег,я очень пож..."
3,"[все, нет, все]",1,все.все,нет.все
4,"[Большой экран, не обнаружил, Популярная новая...",1,Большой экран.Популярная новая модель. это кру...,не обнаружил.Популярная новая модель. это круто.


In [20]:
# соберем плюсы и минусы в один столбец, оставив соответсвующую лейблу комбинацию
adv = df[df['label']==1][['advantages','label']]
adv.rename(columns={"advantages": "comment"}, inplace=True)

disadv = df[df['label']==0][['disadvantages','label']]
disadv.rename(columns={"disadvantages": "comment"}, inplace=True)

df_train = pd.merge(adv, disadv, how='outer')
df_train.drop_duplicates(inplace=True)
df_train.reset_index(inplace=True)

In [220]:
# обучим несколько моделей
for model, i in {'LogisticRegression':LogisticRegression(), 'LinearSVC': LinearSVC(),
                 'RidgeClassifier': RidgeClassifier()}.items():
    pipline = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('transformer', TfidfTransformer()),
            ('classifier', i)
        ])
    score = cross_val_score(pipline, df_train['comment'], df_train['label'])
    print(model, score.mean() )

LogisticRegression 0.8299535840691707
LinearSVC 0.8775433848113356
RidgeClassifier 0.8750915459321764


In [221]:
# LinearSVC выглядит уверенне всего, настроим параметры
params_vectorizer = {
    'vectorizer__max_df' : [0.85, 0.9, 0.95, 1.0],
    'vectorizer__min_df' : [1, 10, 20], 
    'vectorizer__ngram_range' : [(1, 1), (1, 2), (1, 3)],
    'vectorizer__stop_words' : [None, 'english']
}

params_LSVC = {
    'classifier__loss': ['hinge', 'squared_hinge'], 
    'classifier__max_iter': np.arange(100, 1000, 50),
    'classifier__tol': [1e-5, 1e-4, 1e-3, 1e-2],
    'classifier__C': np.arange(0.1, 2, 0.1)}

In [222]:
pipline = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('transformer', TfidfTransformer()),
            ('classifier', LinearSVC())
        ])
grid_cv = RandomizedSearchCV(pipline, {**params_vectorizer,**params_LSVC}, 
                             scoring='accuracy', cv=5, random_state=42, n_iter= 100,n_jobs= -1 )

In [223]:
grid_cv.fit(df_train['comment'], df_train['label'])

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vectorizer', CountVectorizer()),
                                             ('transformer',
                                              TfidfTransformer()),
                                             ('classifier', LinearSVC())]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'classifier__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
                                        'classifier__loss': ['hinge',
                                                             'squared_hinge'],
                                        'classifier__max_iter': array([100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700,
       750, 800, 850, 900, 950]),
                                        'classifier__tol': [1e-05, 0.0001,
                                                            0.001, 0.01],
             

In [224]:
grid_cv.best_estimator_

Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.85, ngram_range=(1, 2))),
                ('transformer', TfidfTransformer()),
                ('classifier',
                 LinearSVC(C=1.7000000000000002, loss='hinge', max_iter=500))])

In [225]:
grid_cv.best_score_

0.8876844696459407

In [228]:
# запишем решение
submission = pd.DataFrame(grid_cv.predict(df_comments['text']), columns=['y'] )
submission['id'] = df_comments.index
submission['y']= submission['y'].apply(lambda x: 'neg' if x == 0 else 'pos')
submission[['id','y']].to_csv('w6_submission.csv', index = False)